In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Interactive Machine Learning - Exercise 03

In this exercise we will learn about cooperative machine learning.
Our goal is it to build a very basic cooperative machine learning user interface and use it to extend our Pokedex model from the last exercise.

The steps you are going to cover are as follows:
* Pretrain our Pokedex model with the original data
* Manually label a small bit of new data
* Train our model on the new data
* Use the model in a cooperative workflow to annotate the rest of the dataset

Please read each exercise carefully before you start coding! You will find a number in the comments before each step of coding you will do. Please refer to these numbers if you have any questions.

## 0. Import the libraries
As always we are providing a list useful packages in the import section below.
Keep in mind that you can import additional libraries at any time and that you do not need to use all the imports if you know another solution for a given task.

In [14]:
import ipywidgets as widgets
import os
import numpy as np
import glob
import random

from IPython.display import Image
from ipywidgets import interact_manual, interact

import keras

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from shutil import copyfile

## 1. Pretrain the model
In this part we are going to pretrain our model on the pokemon images you already know.
To this end we will use the same VGG16 model as last week with the following training procedure:

Preprocessing:
* Imagesize (224,224)
* Vgg16 standard preprocessing from the Keras framework

Datasplit:
* Use 90% of the data to train and 10% to valitdate your results

Training 1:
* Initialize the model with the imagenet weights
* Freeze all convolution layers
* Train the model using the following settings:
 * 5 Epochs
 * Adam Optimizer with default Parameters
 * categorical cross entropy loss
 * Batchsize 32

Training 2:
*  Unfreeze the last two convolutional Blocks
*  Continue training with the following settings:
 * 10 Epochs
 * Adam Optimizer with a learning rate of 0.0001
 * Batchsize of 32

A convolutional Block in the VGG16 architecture consists of 2 to 3 Conv Layers and on Pooling layer.
You can access a models layer directly via `model.layers`.
Read up on how to freeze layers [here](https://keras.io/guides/transfer_learning/), in case you did not use this technique in the last exercise.
Your model should achieve a validation accuracy of close to 100% .

### 1. Load data for pretraining and apply preprocessing

In [15]:
PATH = '/content/gdrive/MyDrive/Colab Data/pokemons/imgs/*'

IMAGE_SIZE = (224, 224)

X = [image.load_img(file, color_mode='rgb', target_size=IMAGE_SIZE, interpolation='nearest') for file in glob.glob(PATH)]
print("Number of pictures of pokemons: {}, and  two of them just for fun of it : \n{}".format(len(X),X[:3]))

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


Number of pictures of pokemons: 347, and  two of them just for fun of it : 
[<PIL.Image.Image image mode=RGB size=224x224 at 0x7F0B75D4C320>, <PIL.Image.Image image mode=RGB size=224x224 at 0x7F0B75D4C390>, <PIL.Image.Image image mode=RGB size=224x224 at 0x7F0B75D4C438>]


In [16]:
X_array = [image.img_to_array(x_, data_format=None, dtype=None) for x_ in X]

print("Number of pictures: {}, \nshape of one of the pictures : ".format(len(X_array),X_array[7].shape))

Number of pictures: 347, 
shape of one of the pictures : 


In [17]:
X_preprocessed =  [preprocess_input(x_) for x_ in X_array]

print("The shape of one of the new preprocessed pictures : \n{} ".format(X_preprocessed[7].shape))

The shape of one of the new preprocessed pictures : 
(224, 224, 3) 


In [18]:
pokemons = ['bulbasaur', 'charmander', 'eevee', 'flareon', 'jolteon', 'pikachu', 'squirtle', 'vaporeon']

# path is equal to ../input/pokemons/imgs/imgs/*
y = [pokemons.index(x.split('/')[7].split('_')[0].lower()) for x in glob.glob(PATH)]
y_encoded = tf.keras.utils.to_categorical(y, 8)
print("The shape of encoded labels: {}, first three of them: \n{}".format(y_encoded.shape, y_encoded[:3]))

The shape of encoded labels: (347, 8), first three of them: 
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]


### 2. Split data into training and test partition

As I understand, we first must ensure that input data is a numpy array

In [19]:
X_pred_ndarray = np.array(X_preprocessed, dtype=np.float32)
print("The shape of images: {}, and type \n{}".format(X_pred_ndarray.shape, type(X_pred_ndarray)))

The shape of images: (347, 224, 224, 3), and type 
<class 'numpy.ndarray'>


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_pred_ndarray, y_encoded, train_size=0.9, random_state=42)



### 3. Define network

In [21]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

base_vgg = VGG16(weights="imagenet", include_top = False, input_shape=(224, 224, 3))
base_vgg.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

### 4. Freeze weights and perform training step 1

In [22]:
base_vgg.trainable = False

base_vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [23]:
input_ = Input(shape=(224, 224, 3))

concat_ = base_vgg(input_, training=False)

vectors_ = Flatten()(concat_)

output_ = Dense(8, activation='softmax', name = 'outupt_eight')(vectors_)

model_vgg = Model(input_, output_)
model_vgg.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
outupt_eight (Dense)         (None, 8)                 200712    
Total params: 14,915,400
Trainable params: 200,712
Non-trainable params: 14,714,688
_________________________________________________________________


In [24]:
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics='acc')

history = model_vgg.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32)

Epoch 1/5
10/10 [==============================] - 3s 274ms/step - loss: 18.4990 - acc: 0.4647 - val_loss: 6.1892 - val_acc: 0.7429
Epoch 2/5
10/10 [==============================] - 1s 116ms/step - loss: 2.1833 - acc: 0.9071 - val_loss: 5.2614 - val_acc: 0.8000
Epoch 3/5
10/10 [==============================] - 1s 115ms/step - loss: 0.0412 - acc: 0.9968 - val_loss: 6.6279 - val_acc: 0.8286
Epoch 4/5
10/10 [==============================] - 1s 117ms/step - loss: 0.0993 - acc: 0.9936 - val_loss: 7.0155 - val_acc: 0.8286
Epoch 5/5
10/10 [==============================] - 1s 115ms/step - loss: 1.4166e-06 - acc: 1.0000 - val_loss: 6.9978 - val_acc: 0.8286


### 5. Unfreeze weights and perform training step 2

Task is: 
* Unfreeze the last two convolutional Blocks
* Continue training with the following settings:
    * 10 Epochs
    * Adam Optimizer with a learning rate of 0.0001
    * Batchsize of 32

Base model has 19 layers. The last one in it is MaxPooling and two before MaxPooling are Conv2d - exactly what we must set to trainable

In [25]:
base_vgg.trainable = True
for layer in base_vgg.layers[0:16]:
    layer.trainable =  False
    
base_vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [26]:
model_vgg.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics='acc')

model_vgg.summary()

history = model_vgg.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
outupt_eight (Dense)         (None, 8)                 200712    
Total params: 14,915,400
Trainable params: 4,920,328
Non-trainable params: 9,995,072
_________________________________________________________________
Epoch 1/10
10/10 [==============================] - 2s 152ms/step - loss: 1.0813e-07 - acc: 1.0000 - val_loss: 8.9200 - val_acc: 0.8286
Epoch 2/10
10/10 [==============================] - 1s 129ms/step - loss: 6.4954e-09 - acc: 1.000

## 2. Pretrain the model
Now that we have our initial model we are going to extend it with some more pokemon.
[Here](https://megastore.uni-augsburg.de/get/OxpI3M_JyU/) you will find roughly 6000 images of the following Pokemon:
* Blastoise
* Charizard
* Charmeleon
* Ivysaur
* Venusaur
* Wartortle

Unfortunately images are not labeled yet. To speed things up a bit we are only going to label a small part of the data ourselves, and then build a model to help us doing the rest.
(Actually this will probably not be faster, but more fun anyway :) ).
In your project directory you will find a 'data_labled' folder, which we will use to store the labeled data.
This time we will use the folder structure to create our labels and train / validation partitions.
Inside the folder you will therefore find a 'train' and an 'val' folder, each of them containing subfolders for each class.

In the following step you should at first manually pick at least 5 examples per class and copy them from the 'data' folder to the train partition of the 'data_labeled' folder.
To then take full advantage of the current way the data is structured, we will use keras data generators in combination with the `flow_from_directory` to dynamically read the input data and feed it to our model.
You can find an example of such data generators [here](https://keras.io/api/preprocessing/image/#flowfromdirectory-method).

Specifically we are going to write a function `train_loop()` which creates two data generators (one for training and one for validation) and trains a model for the new Images on features extracted from our current Pokexedx model.
To this end you can simply rebuild the structure of the original model, but replace the number of output classes.
To load the weights you can then use the following code snippet:
`model.layers[-1]._name = 'new_output'`</br>
`model.load_weights(weight_path, by_name=True)`</br>

Freeze all layers but the dense layers, we will only need those and want to speed up the training process a bit.

### Saving model(for fun) and its weights

In [27]:
model_vgg.save('model_vgg-pokemons_on_imgs.h5')

model_vgg.save_weights('model_vgg-pokemons_on_imgs_weights.hdf5')

### 6. Copy at least 5 images per class from the data folder to the correct partition in the data_labeled folder

*DONE!*

### 7. Function train_loop()

In [28]:
def train_loop():
    keras.backend.clear_session()
    tf.random.set_seed(42)
    np.random.seed(42)

    # 8. Build model
    input_ = Input(shape=(224, 224, 3))

    concat_ = base_vgg(input_, training=False)

    vectors_ = Flatten()(concat_)

    output_ = Dense(6, activation='softmax', name = 'output_six')(vectors_)

    model = Model(input_, output_)
    """
    print("Model's summary BEFORE loading weights:") 
    model.summary()
    print("BASE Model's summary BEFORE loading weights:")
    base_vgg.summary()
    """
    # 9. Load weights
    print('LOADING WEIGHTS . . .')
    model.load_weights('model_vgg-pokemons_on_imgs_weights.hdf5', by_name = True)
    """
    print("Model's summary AFTER loading weights: ")
    model.summary()
    print("BASE Model's summary AFTER loading weights:")
    base_vgg.summary()
    """
    # 10. Build data generators
    train_data_dir = '/content/gdrive/MyDrive/Colab Data/pokemons/data_labeled/train'
    val_data_dir = '/content/gdrive/MyDrive/Colab Data/pokemons/data_labeled/val'

    train_datagen = image.ImageDataGenerator()
    val_datagen = image.ImageDataGenerator()

    train_generator = train_datagen.flow_from_directory(
        directory = train_data_dir,
        target_size=(224, 224),
        batch_size=16)

    validation_generator = val_datagen.flow_from_directory(
        directory = val_data_dir,
        target_size=(224, 224),
        batch_size=16)
    
    # 11. Fit the model to the data for a few epochs
    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics='acc')
    
    model.fit_generator(train_generator, validation_data=validation_generator, epochs = 10)

### 12. Call train loop

In [29]:
train_loop()

LOADING WEIGHTS . . .
Found 30 images belonging to 6 classes.
Found 90 images belonging to 6 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
2/2 [==============================] - 47s 23s/step - loss: 31.5176 - acc: 0.1667 - val_loss: 22.9393 - val_acc: 0.2333
Epoch 2/10
2/2 [==============================] - 1s 420ms/step - loss: 2.6575 - acc: 0.8333 - val_loss: 21.3930 - val_acc: 0.2889
Epoch 3/10
2/2 [==============================] - 1s 410ms/step - loss: 0.2865 - acc: 0.9667 - val_loss: 18.1234 - val_acc: 0.2889
Epoch 4/10
2/2 [==============================] - 1s 405ms/step - loss: 8.7946e-04 - acc: 1.0000 - val_loss: 15.5587 - val_acc: 0.3222
Epoch 5/10
2/2 [==============================] - 1s 412ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 13.8663 - val_acc: 0.3889
Epoch 6/10
2/2 [==============================] - 1s 406ms/step - loss: 3.8415e-05 - acc: 1.0000 - val_loss: 12.9300 - val_acc: 0.4333
Epoch 7/10
2/2 [============

## 3. Interactive UI

In this part of the exercise we are going to put our pretrained model to good use by employing it in a cooperative workflow.
To this end we gonna build a minimal cooperative machine learning using interface in this python notebook.
Our user interface will consist of the following components:

* (optional) A progressbar to keep to motivation up
* A slider to set a high confidence threshold
* A slider to set the mid confidence threshold
* Some radio buttons to choose the label
* A button to save the annotation and label and show the next image
* A button to retrain our model
* A button to use our model to predict our dataset

The final our UI should look a like this:

![img](https://hcm-lab.de/cloud/index.php/s/ak3txGXepnt9NxS/preview)

The 'retrain' button should call the `train_loop()`  function from before to retrain the model on all labeled data.
The 'predict' button should create a list of predictions for all unlabeled images.
All predictions that are above the high confidence threshold, set by the respective slider, should be automatically accepted as correct label and copied to the respective folders in the training data folder.
Additionally you should implement a garbage label to delete unfitting images.
Potential reasons to consider an Image as garbage are if no Pokemon is visible, too many Pokemon are visible, non of the Pokemon we want to train are visible, the Imagefile is broken etc.
When you are pressing the 'next' button the current image should be copied to the right folder in the training dataset, depending on the current value of the radio button.
Afterwards the next image should be chosen from all predicted images, where the confidence is greater or equal than the value set by the mid_threshold slider.
The current value of the radiobutton should then be set to the prediction for this respective image.
Optionally you can also implement a progressbar to track your progress for you annotations.

You can use the ipywidgets library to create the UI.
You can find an IPython tutorial [here](https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6) and the api documentation [here](https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6).
Note, that Pycharm might not play well with the the widgets in all scenarios. It's best to view them in the browser by visting: http://localhost:8888 after you started your notebook.

### 13. Build UI

#### Methods to create widgets

In [47]:
def create_predict_button():    
  button_predict = widgets.Button(description='Predict', button_style='success')
  button_predict.on_click(on_predict)
  return button_predict

def create_retrain_button():
  button_retrain = widgets.Button(description='Retrain', button_style='danger')
  button_retrain.on_click(on_retrain)
  return button_retrain

def create_next_button():
  button_next = widgets.Button(description='Next', button_style='success')
  button_next.on_click(on_next)
  return button_next

labels = widgets.RadioButtons(options=['Blastoise', 'Charizard', 'Charmeleon', 'Ivysaur', 'Venusaur', 'Wartortle'],  
                                description='Labels:', disabled=False)

def create_labels_radio():
  return labels

high_conf = widgets.IntSlider(value=80, min=0, max=100, step=5, description='High Conf:', disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='d')

def create_high_conf():
  return high_conf

mid_conf = widgets.IntSlider(value=50, min=0, max=100, step=5, description='Mid Conf:', disabled=False, continuous_update=False,
    orientation='horizontal', readout=True, readout_format='d')

def create_mid_conf():
  return mid_conf

path = '/content/gdrive/MyDrive/Colab Data/pokemons/data/'

def create_image():
  """file=os.listdir(path)
  display(Image(path+file))"""
  
  file = open(path, "rb")
  image = file.read()
  return widgets.Image(value=image)


#### Collable methods from widgets

In [48]:
def on_predict(r):
  print('Predicting ...', r)

def on_retrain(r):
  train_loop()

def on_next(label=''):
  print('Next ...', labels.value)

def on_label():
  print('{} ...')

In [ ]:
path = '/content/gdrive/MyDrive/Colab Data/pokemons/data/*'

X = [image.load_img(file, target_size=IMAGE_SIZE, interpolation='nearest') for file in glob.glob(path)]
X_array = [image.img_to_array(x_, data_format=None, dtype=None) for x_ in X]
X_preprocessed =  [preprocess_input(x_) for x_ in X_array]
X_test = np.array(X_preprocessed, dtype=np.float32)

#### Layout

In [49]:
from ipywidgets import GridspecLayout

grid = GridspecLayout(9, 4, height='600px')
grid[1, 0] = create_high_conf()
grid[2, 0] = create_mid_conf()
grid[3, 0] = create_labels_radio()
grid[5, 0] = create_next_button()
grid[7, 0] = create_retrain_button()
grid[8, 0] = create_predict_button()
grid[:, 1:] = create_image()

display(grid)

GridspecLayout(children=(IntSlider(value=80, continuous_update=False, description='High Conf:', layout=Layout(…

## 4. repeat(annotate, train, predict)
After you are done creating the UI, we are now going to label the whole dataset together with our model.
To this end use your model to predict and improve iteratively in the following manner:

Set the high confidence slider to a value greater or equal than 0.95 and the mid confidence slider to at least 0.8

Repeat 3 times:

* Call automatic prediction
* Check images that have been above the maximum confidence threshold manually by looking at the content of the respective folders. Make corrections if necessary.
* Annotate remaining images that have been over the mid confidence score
* Retrain you model

Do you notice any change in the amount of images you have to annotate each time?

Repeat till all data is annotated:

* Call automatic prediction
* Annotate remaining images that have been over the mid confidence score
* Retrain you model
* Adjust both confidence scores based on how much you trust your model

Describe your subjective impression of the annotation process. Did you have the feeling, that the cooperative workflow is helpful?

ANSWER:  It is a very helpful tool to manually annotate images that model cannot assign class with desired confidence.